In [1]:
devtools::install_github("cellgeni/sceasy")
install.packages('anndata')
library(anndata)

Skipping install of 'sceasy' from a github remote, the SHA1 (c1c0bf97) has not changed since last install.
  Use `force = TRUE` to force installation

Installing package into ‘/home/jupyter/libb’
(as ‘lib’ is unspecified)



In [2]:
library(sceasy)
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(H5weaver))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(lme4))
suppressPackageStartupMessages(library(MAST))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(parallel)
library(DESeq2)
options("mc.cores"=50)
library(ComplexHeatmap)
library(circlize)


Loading required package: reticulate

Loading required package: grid

ComplexHeatmap version 2.18.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))
! pheatmap() has been masked by ComplexHeatmap::pheatmap(). Most of the arguments
   in the original pheatmap() are identically supported in the new function. You 
   can still use the original function by explicitly calling pheatmap:

In [3]:
ImmuneType<-read.csv('SLP_Immunotype_Assignments.csv')
colnames(ImmuneType)<-c('ImmuneType','subject.subjectGuid')
ImmuneType$ImmuneType<-as.character(ImmuneType$ImmuneType)

In [4]:
meta_data=read.csv('/home//jupyter/BRI_Analysis/scRNA/hise_meta_data_2024-01-23_fixed.csv')
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 1 Day 0'))

In [5]:
meta_data_subset<-left_join(meta_data_subset,ImmuneType,)

Joining with `by = join_by(subject.subjectGuid)`


In [6]:
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)


In [ ]:
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  


dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  0+ImmuneType)
dds <- DESeq(dds,parallel=FALSE)

res_ImmuneType1 <- data.frame(results(dds,contrast=list(c("ImmuneType1"), 
                      c("ImmuneType2","ImmuneType3","ImmuneType4")),
        listValues=c(1, -1/3)))
res_ImmuneType2 <- data.frame(results(dds,contrast=list(c("ImmuneType2"), 
                      c("ImmuneType1","ImmuneType3","ImmuneType4")),
        listValues=c(1, -1/3)))    
res_ImmuneType3 <- data.frame(results(dds,contrast=list(c("ImmuneType3"), 
                      c("ImmuneType2","ImmuneType1","ImmuneType4")),
        listValues=c(1, -1/3)))
res_ImmuneType4 <- data.frame(results(dds,contrast=list(c("ImmuneType4"), 
                      c("ImmuneType2","ImmuneType3","ImmuneType1")),
        listValues=c(1, -1/3)))
res_ImmuneType1$ImmuneType<-"1"
res_ImmuneType2$ImmuneType<-"2"
res_ImmuneType3$ImmuneType<-"3"
res_ImmuneType4$ImmuneType<-"4"
res_ImmuneType1$Gene<-rownames(res_ImmuneType1)
res_ImmuneType2$Gene<-rownames(res_ImmuneType2)
res_ImmuneType3$Gene<-rownames(res_ImmuneType3)
res_ImmuneType4$Gene<-rownames(res_ImmuneType4)
res<-do.call(rbind,list(res_ImmuneType1,res_ImmuneType2,res_ImmuneType3,res_ImmuneType4))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInOtherImmueType',log2FoldChange>0~'HigherInThisImmuneType'))
res$Year<-"Y1D0"

write.csv(res,paste0("DESeq2_ImmuneType/",celltype,".csv"))

}



In [8]:
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  


dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  0+ImmuneType)
dds <- DESeq(dds,parallel=FALSE)

res_ImmuneType1 <- data.frame(results(dds,contrast=list(c("ImmuneType1"), 
                      c("ImmuneType2","ImmuneType3","ImmuneType4")),
        listValues=c(1, -1/3)))
res_ImmuneType2 <- data.frame(results(dds,contrast=list(c("ImmuneType2"), 
                      c("ImmuneType1","ImmuneType3","ImmuneType4")),
        listValues=c(1, -1/3)))    
res_ImmuneType3 <- data.frame(results(dds,contrast=list(c("ImmuneType3"), 
                      c("ImmuneType2","ImmuneType1","ImmuneType4")),
        listValues=c(1, -1/3)))
res_ImmuneType4 <- data.frame(results(dds,contrast=list(c("ImmuneType4"), 
                      c("ImmuneType2","ImmuneType3","ImmuneType1")),
        listValues=c(1, -1/3)))
res_ImmuneType1$ImmuneType<-"1"
res_ImmuneType2$ImmuneType<-"2"
res_ImmuneType3$ImmuneType<-"3"
res_ImmuneType4$ImmuneType<-"4"
res_ImmuneType1$Gene<-rownames(res_ImmuneType1)
res_ImmuneType2$Gene<-rownames(res_ImmuneType2)
res_ImmuneType3$Gene<-rownames(res_ImmuneType3)
res_ImmuneType4$Gene<-rownames(res_ImmuneType4)
res<-do.call(rbind,list(res_ImmuneType1,res_ImmuneType2,res_ImmuneType3,res_ImmuneType4))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInOtherImmueType',log2FoldChange>0~'HigherInThisImmuneType'))
res$Year<-"Y1D0"

write.csv(res,paste0("DESeq2_ImmuneType_only/",celltype,".csv"))

}



[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing

